# Hyper-parameter Tuning with Ray-Tune Tutorial
This tutorial helps you tune hyper-parameters (e.g., learning rate, batch size, number of latent dimensions, etc) in BEMB.

We will be using the `ray` library, which enables parallelization, to tune hyper-parameters.

For more details regarding using Ray to tune hyper-parameters of models (especially PyTorch lightning models), please refer to this [tutorial](https://docs.ray.io/en/latest/ray-core/examples/using-ray-with-pytorch-lightning.html).

Author: Tianyu Du
Date: July. 27, 2022

In [1]:
import argparse
import os
import time
from datetime import datetime


from typing import List
import numpy as np
import pandas as pd
import torch
from torch_choice.data import ChoiceDataset
from bemb.model import LitBEMBFlex
from bemb.utils.run_helper import run

from torch.utils.data import DataLoader

from bemb.model import LitBEMBFlex

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from ray import tune
from ray.tune import CLIReporter
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler

In [65]:
from torch_choice.data.utils import create_data_loader

The idea behind tuning hyper-parameters is to find the best configuration of the model among a space of hyper-parameters.

## Simulate Datasets

I use the simulated datasets from the *simulation* tutorial (refer to the `tutorials/simulation/simulation.ipynb` notebook).

The `simulate_dataset` method returns a list of three `ChoiceDataset` corresponding to the train/validation/test dataset.

In [2]:
# def simulate_dataset() -> List[ChoiceDataset]:
num_users = 1500
num_items = 50
data_size = 1000
user_index = torch.LongTensor(np.random.choice(num_users, size=data_size))
Us = np.arange(num_users)
Is = np.sin(np.arange(num_users) / num_users * 4 * np.pi)
Is = (Is + 1) / 2 * num_items
Is = Is.astype(int)

PREFERENCE = dict((u, i) for (u, i) in zip(Us, Is))

item_index = torch.LongTensor(np.random.choice(num_items, size=data_size))

for idx in range(data_size):
    if np.random.rand() <= 0.5:
        item_index[idx] = PREFERENCE[int(user_index[idx])]

# df = pd.DataFrame(data={'item': item_index, 'user': user_index}).groupby(['item', 'user']).size().rename('size').reset_index()
# df = df.pivot('item', 'user', 'size').fillna(0.0)

user_obs = torch.zeros(num_users, num_items)
user_obs[torch.arange(num_users), Is] = 1

item_obs = torch.eye(num_items)

dataset = ChoiceDataset(user_index=user_index, item_index=item_index, user_obs=user_obs, item_obs=item_obs)

idx = np.random.permutation(len(dataset))
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
train_idx = idx[:train_size]
val_idx = idx[train_size: train_size + val_size]
test_idx = idx[train_size + val_size:]

dataset_list = [dataset[train_idx], dataset[val_idx], dataset[test_idx]]
    # return dataset_list

No `session_index` is provided, assume each choice instance is in its own session.


In [3]:
# dataset_list = simulate_dataset()
dataset_list

[ChoiceDataset(label=[], item_index=[800], user_index=[800], session_index=[800], item_availability=[], user_obs=[1500, 50], item_obs=[50, 50], device=cpu),
 ChoiceDataset(label=[], item_index=[100], user_index=[100], session_index=[100], item_availability=[], user_obs=[1500, 50], item_obs=[50, 50], device=cpu),
 ChoiceDataset(label=[], item_index=[100], user_index=[100], session_index=[100], item_availability=[], user_obs=[1500, 50], item_obs=[50, 50], device=cpu)]

## Model

In [33]:
num_samples = 3
num_epochs = 50

In [70]:
def train_with_specific_params(hparams):
    bemb = LitBEMBFlex(
        learning_rate=hparams['learning_rate'],  # set the learning rate, feel free to play with different levels.
        pred_item=True,  # let the model predict item_index, don't change this one.
        num_seeds=32,  # number of Monte Carlo samples for estimating the ELBO.
        utility_formula='theta_user * alpha_item',  # the utility formula.
        num_users=num_users,
        num_items=num_items,
        num_user_obs=dataset.user_obs.shape[1],
        num_item_obs=dataset.item_obs.shape[1],
        # whether to turn on obs2prior for each parameter.
        obs2prior_dict={'theta_user': hparams['obs2prior'], 'alpha_item': hparams['obs2prior']},
        # the dimension of latents, since the utility is an inner product of theta and alpha, they should have
        # the same dimension.
        coef_dim_dict={'theta_user': hparams['latent_dim'], 'alpha_item': hparams['latent_dim']}
    )

    trainer = pl.Trainer(
        max_epochs=10,
        check_val_every_n_epoch=1,
        log_every_n_steps=1,
        gpus=0,
        progress_bar_refresh_rate=0,
        auto_lr_find=False,
        logger=TensorBoardLogger(save_dir='./', name='', version='.'),
        callbacks=[TuneReportCallback({'val_ll': 'val_ll', 'val_acc': 'val_acc'}, on='validation_end'),
                   EarlyStopping(monitor='val_acc', patience=30, mode='max')])

    # find an appropriate learning rate.
    # trainer.tune(bemb,
    #              train_dataloaders=DataLoader(dataset_list[0]),
    #              val_dataloaders=DataLoader(dataset_list[1]))
                #  test_dataloaders=dataset_list[2])
    trainer.fit(bemb, train_dataloaders=create_data_loader(dataset_list[0]), val_dataloaders=create_data_loader(dataset_list[1]))

In [71]:
config = {
    'learning_rate': tune.choice([0.01, 0.03, 0.1, 0.3]),
    'latent_dim': tune.choice([10, 20, 50, 100]),
    'obs2prior': tune.choice([True, False])
}

# scheduler = ASHAScheduler(max_t=num_epochs, grace_period=1, reduction_factor=2)
scheduler = None

reporter = CLIReporter(parameter_columns=list(config.keys()),
                        metric_columns=['val_ll', 'val_acc'])

In [72]:
analysis = tune.run(
    tune.with_parameters(train_with_specific_params),
    # metric='val_ll',
    # mode='min',
    # NOTE: I am fixing the GPU support now, set it to 0.
    resources_per_trial={'cpu': 4, 'gpu': 0},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter
)


2022-07-27 01:35:23,380	WARNING tune.py:668 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


== Status ==
Current time: 2022-07-27 01:35:23 (running for 00:00:00.17)
Memory usage on this node: 5.7/125.5 GiB
Using FIFO scheduling algorithm.
Resources requested: 4.0/16 CPUs, 0/1 GPUs, 0.0/76.98 GiB heap, 0.0/36.98 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/tianyudu/ray_results/train_with_specific_params_2022-07-27_01-35-23
Number of trials: 3/3 (2 PENDING, 1 RUNNING)
+----------------------------------------+----------+---------------------+-----------------+--------------+-------------+
| Trial name                             | status   | loc                 |   learning_rate |   latent_dim | obs2prior   |
|----------------------------------------+----------+---------------------+-----------------+--------------+-------------|
| train_with_specific_params_0e85e_00000 | RUNNING  | 192.168.0.158:14653 |            0.1  |          100 | True        |
| train_with_specific_params_0e85e_00001 | PENDING  |                     |            0.3  |           10 | Tru

(train_with_specific_params pid=14653) /home/tianyudu/anaconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
(train_with_specific_params pid=14653)   rank_zero_deprecation(
(train_with_specific_params pid=14653) GPU available: False, used: False
(train_with_specific_params pid=14653) TPU available: False, using: 0 TPU cores
(train_with_specific_params pid=14653) IPU available: False, using: 0 IPUs
(train_with_specific_params pid=14653) HPU available: False, using: 0 HPUs
(train_with_specific_params pid=14653) /home/tianyudu/anaconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning

(train_with_specific_params pid=14694) BEMB: utility formula parsed:
(train_with_specific_params pid=14694) [{'coefficient': ['theta_user', 'alpha_item'], 'observable': None}]
(train_with_specific_params pid=14692) BEMB: utility formula parsed:
(train_with_specific_params pid=14692) [{'coefficient': ['theta_user', 'alpha_item'], 'observable': None}]


(train_with_specific_params pid=14692) /home/tianyudu/anaconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
(train_with_specific_params pid=14692)   rank_zero_deprecation(
(train_with_specific_params pid=14692) GPU available: False, used: False
(train_with_specific_params pid=14692) TPU available: False, using: 0 TPU cores
(train_with_specific_params pid=14692) IPU available: False, using: 0 IPUs
(train_with_specific_params pid=14692) HPU available: False, using: 0 HPUs
(train_with_specific_params pid=14692) /home/tianyudu/anaconda3/envs/ml/lib/python3.8/site-packages/pytorch_lightning

Result for train_with_specific_params_0e85e_00000:
  date: 2022-07-27_01-35-27
  done: false
  experiment_id: 3d450a95dec44141853a33437c188b91
  hostname: aurora
  iterations_since_restore: 1
  node_ip: 192.168.0.158
  pid: 14653
  time_since_restore: 2.4849677085876465
  time_this_iter_s: 2.4849677085876465
  time_total_s: 2.4849677085876465
  timestamp: 1658910927
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0e85e_00000
  val_acc: 0.03
  val_ll: -18.660343192787114
  warmup_time: 0.001985788345336914
  
Result for train_with_specific_params_0e85e_00001:
  date: 2022-07-27_01-35-27
  done: false
  experiment_id: 1cc669fd41a743e7b52da84c466a60ce
  hostname: aurora
  iterations_since_restore: 1
  node_ip: 192.168.0.158
  pid: 14692
  time_since_restore: 0.8729078769683838
  time_this_iter_s: 0.8729078769683838
  time_total_s: 0.8729078769683838
  timestamp: 1658910927
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 0e85e_00001
  val_acc: 0.03
  val_l

2022-07-27 01:35:36,854	INFO tune.py:747 -- Total run time: 13.49 seconds (13.37 seconds for the tuning loop).


Result for train_with_specific_params_0e85e_00000:
  date: 2022-07-27_01-35-36
  done: true
  experiment_id: 3d450a95dec44141853a33437c188b91
  experiment_tag: 0_latent_dim=100,learning_rate=0.1000,obs2prior=True
  hostname: aurora
  iterations_since_restore: 10
  node_ip: 192.168.0.158
  pid: 14653
  time_since_restore: 11.668215990066528
  time_this_iter_s: 0.9347765445709229
  time_total_s: 11.668215990066528
  timestamp: 1658910936
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: 0e85e_00000
  val_acc: 0.15
  val_ll: -13.605984133323654
  warmup_time: 0.001985788345336914
  
== Status ==
Current time: 2022-07-27 01:35:36 (running for 00:00:13.37)
Memory usage on this node: 6.2/125.5 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/76.98 GiB heap, 0.0/36.98 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /home/tianyudu/ray_results/train_with_specific_params_2022-07-27_01-35-23
Number of trials: 3/3 (3 TERMINATED)
+----------